In [30]:
#Import necessary libraries
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
import gradio as gr
import os


In [31]:
# Define templates for conversation and document-based prompts
template = """You are a helpful assistant.
Current conversation:
{history}
Human: {input}
AI Assistant:"""

document_template = """Answer the question based only on the following context:
{context}

Question: {question}
"""


In [32]:
# Creates a prompt template using PromptTemplate, which takes history and input as variables 
# and formats them according to the template string.
PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)
# Create a prompt template for conversation
llm = ChatGroq(
    temperature=0, # Set the randomness of the model's responses
    groq_api_key='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', # Get the API key from environment variables
    model_name="llama-3.1-8b-instant" # Specify the model name
)
memory = ConversationBufferMemory()   # Initialize memory to store conversation history
conversation = ConversationChain(llm=llm, memory=memory, verbose=True, prompt=PROMPT)


In [33]:
# Create a conversation chain with LLM, memory, and prompt
# Function to load and retrieve documents
def load_and_retrieve(file):
    file = PyPDFLoader(file).load() # Load the PDF file
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200) # Split the document into chunks
    chunks = splitter.split_documents(file)  # Split the loaded document into chunks
    embeddings = HuggingFaceEmbeddings() # Generate embeddings for the document chunks
    vectorstore = FAISS.from_documents(documents=chunks, embedding=embeddings) # Store the embeddings in a FAISS vector database
    return vectorstore.as_retriever()  # Return a retriever for querying the vector database


In [34]:
# Function to create a RAG (Retrieval-Augmented Generation) chain
def rag_chain(file, question):
    retriever = load_and_retrieve(file) # Load and retrieve document chunks
    prompt = ChatPromptTemplate.from_template(document_template)  # Create a chat prompt template from the document template
    chain = (
        {"context": retriever, "question": RunnablePassthrough()} # Pass the context and question through the chain
        | prompt  # Apply the prompt template
        | llm  # Use the language model to generate a response
        | StrOutputParser()  # Parse the output as a string
    )
    response = chain.invoke(question) # Invoke the chain with the question
    return response  # Return the generated response


In [35]:
# Function to handle chat interactions
def handle_chat(file, question):  
    if file:
        response = rag_chain(file, question)   # Use the RAG chain if a file is provided
    else:
        response = conversation.predict(input=question)  # Use the conversation chain if no file is provided
    return response


In [36]:
# Example question
question = "What is good in being a programmer"


In [37]:
# Print response
file = None  # Replace with your file path if needed
response = handle_chat(file, question)
print("Response:", response)




> Entering new ConversationChain chain...
Prompt after formatting:
You are a helpful assistant.
Current conversation:

Human: What is good in being a programmer
AI Assistant:

> Finished chain.
Response: Being a programmer can be incredibly rewarding and offers many benefits. Here are some of the advantages of being a programmer:

1. **Creative Freedom**: As a programmer, you have the ability to create something from scratch, bringing your ideas to life. You can design, develop, and implement solutions to real-world problems, which can be very fulfilling.

2. **Constant Learning**: The field of programming is constantly evolving, with new technologies, frameworks, and languages emerging all the time. This means that programmers must stay up-to-date with the latest developments, which can be a fun and challenging experience.

3. **Good Compensation**: Programmers are generally well-compensated, with median salaries ranging from $60,000 to over $100,000 depending on the location, exper